In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from helpers.preproccesing_tabnet_attack import get_weights, normalize, get_bounds

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.datasets import fetch_openml

import pandas as pd
import numpy as np
np.random.seed(0)

# Pytorch
import torch
import torch.nn as nn
from torch.autograd import Variable

import tqdm
from tqdm import tqdm
from tqdm import tqdm


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

# Get Credit-g Dataset

In [2]:
dataset = 'credit-g'
dataset_name = 'credit-g'
target = 'target' 

dataset = fetch_openml(dataset)

train = pd.DataFrame(data= np.c_[dataset['data'], dataset[target]],
                  columns= dataset['feature_names'] + [target]) 

 # Renaming target for training later
train[target] = train[target].apply(lambda x : 0.0 if x == 'bad' or x == 0.0 else 1)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/datasets/_openml.py:421: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  " {version}.".format(name=name, version=res[0]["version"])


# Split Data

In [3]:
train_set = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(train.shape[0],))

train_indices = train[train_set=="train"].index
valid_indices = train[train_set=="valid"].index
test_indices = train[train_set=="test"].index


# Preproccessing Label encode categorical features.

In [4]:
nunique = train.nunique()
types = train.dtypes

categorical_columns = []
categorical_dims =  {}
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = train[col].fillna("VV_likely")
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)

checking_status 4
duration 33
credit_history 5
purpose 10
credit_amount 921
savings_status 5
employment 5
installment_commitment 4
personal_status 4
other_parties 3
residence_since 4
property_magnitude 4
age 53
other_payment_plans 3
housing 3
existing_credits 4
job 4
num_dependents 2
own_telephone 2
foreign_worker 2
target 2


# Define categorical features for categorical embeddings

In [5]:
# unused_feat = ['Set']

unused_feat = []

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

# Network parameters

In [6]:
tabnet_params = {"cat_idxs":cat_idxs,
                 "cat_dims":cat_dims,
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-3),
                 "scheduler_params":{"step_size":20, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
                      )



/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [7]:
SEED = 0
# Compute the bounds for clipping
bounds = get_bounds(train)

# Normalize the data
#scaler, train, bounds = normalize(train, target, features, bounds)

# Compute the weights modelizing the expert's knowledge
weights = get_weights(train, target)

# Training

In [8]:
X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

X_test = train[features].values[test_indices]
y_test = train[target].values[test_indices]

In [9]:
max_epochs = 1000 if not os.getenv("CI", False) else 2

In [10]:
from pytorch_tabnet.augmentations import ClassificationSMOTE
aug = ClassificationSMOTE(p=0.2)

In [11]:
## This illustrates the warm_start=False behaviour
save_history = []
for _ in range(2):
    clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_name=['train', 'valid'],
        eval_metric=['auc'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False,
        augmentations=aug, #aug, None
    )
    save_history.append(clf.history["valid_auc"])

assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 1.12169 | train_auc: 0.49059 | valid_auc: 0.41205 |  0:00:00s
epoch 1  | loss: 1.11225 | train_auc: 0.52627 | valid_auc: 0.43017 |  0:00:00s
epoch 2  | loss: 1.02725 | train_auc: 0.53272 | valid_auc: 0.45416 |  0:00:00s
epoch 3  | loss: 0.95978 | train_auc: 0.5317  | valid_auc: 0.46482 |  0:00:00s
epoch 4  | loss: 0.95134 | train_auc: 0.52825 | valid_auc: 0.46802 |  0:00:00s
epoch 5  | loss: 0.95533 | train_auc: 0.53019 | valid_auc: 0.47228 |  0:00:00s
epoch 6  | loss: 0.96258 | train_auc: 0.53373 | valid_auc: 0.48134 |  0:00:00s
epoch 7  | loss: 0.91441 | train_auc: 0.53673 | valid_auc: 0.49627 |  0:00:00s
epoch 8  | loss: 0.93052 | train_auc: 0.5393  | valid_auc: 0.50533 |  0:00:00s
epoch 9  | loss: 0.85057 | train_auc: 0.54247 | valid_auc: 0.51812 |  0:00:01s
epoch 10 | loss: 0.84792 | train_auc: 0.54479 | valid_auc: 0.52665 |  0:00:01s
epoch 11 | loss: 0.80767 | train_auc: 0.54851 | valid_auc: 0.52985 |  0:00:01s
epoch 12 | loss: 0.82005 | train_auc: 0.54813 | vali

epoch 106| loss: 0.65534 | train_auc: 0.64    | valid_auc: 0.60181 |  0:00:10s
epoch 107| loss: 0.65302 | train_auc: 0.6414  | valid_auc: 0.60128 |  0:00:11s
epoch 108| loss: 0.65729 | train_auc: 0.64237 | valid_auc: 0.59808 |  0:00:11s
epoch 109| loss: 0.65786 | train_auc: 0.64352 | valid_auc: 0.59915 |  0:00:11s
epoch 110| loss: 0.6498  | train_auc: 0.64193 | valid_auc: 0.59488 |  0:00:11s
epoch 111| loss: 0.66234 | train_auc: 0.6417  | valid_auc: 0.59808 |  0:00:11s
epoch 112| loss: 0.63517 | train_auc: 0.64205 | valid_auc: 0.59648 |  0:00:11s
epoch 113| loss: 0.65684 | train_auc: 0.64161 | valid_auc: 0.59701 |  0:00:11s
epoch 114| loss: 0.67401 | train_auc: 0.64109 | valid_auc: 0.60181 |  0:00:11s
epoch 115| loss: 0.63866 | train_auc: 0.64122 | valid_auc: 0.60501 |  0:00:11s
epoch 116| loss: 0.64437 | train_auc: 0.64264 | valid_auc: 0.60608 |  0:00:11s
epoch 117| loss: 0.63941 | train_auc: 0.64261 | valid_auc: 0.60661 |  0:00:12s
epoch 118| loss: 0.65528 | train_auc: 0.6429  | vali

epoch 211| loss: 0.59173 | train_auc: 0.71481 | valid_auc: 0.63273 |  0:00:21s
epoch 212| loss: 0.59153 | train_auc: 0.71506 | valid_auc: 0.63273 |  0:00:21s
epoch 213| loss: 0.60921 | train_auc: 0.71639 | valid_auc: 0.63539 |  0:00:21s
epoch 214| loss: 0.61612 | train_auc: 0.71842 | valid_auc: 0.63806 |  0:00:21s
epoch 215| loss: 0.59058 | train_auc: 0.71956 | valid_auc: 0.63539 |  0:00:21s
epoch 216| loss: 0.60639 | train_auc: 0.7207  | valid_auc: 0.6338  |  0:00:21s
epoch 217| loss: 0.60112 | train_auc: 0.72135 | valid_auc: 0.63486 |  0:00:21s
epoch 218| loss: 0.59969 | train_auc: 0.72294 | valid_auc: 0.63806 |  0:00:21s
epoch 219| loss: 0.60936 | train_auc: 0.72133 | valid_auc: 0.63593 |  0:00:21s
epoch 220| loss: 0.60694 | train_auc: 0.72077 | valid_auc: 0.63699 |  0:00:21s

Early stopping occurred at epoch 220 with best_epoch = 200 and best_valid_auc = 0.64819
epoch 0  | loss: 1.12169 | train_auc: 0.49059 | valid_auc: 0.41205 |  0:00:00s


/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


epoch 1  | loss: 1.11225 | train_auc: 0.52627 | valid_auc: 0.43017 |  0:00:00s
epoch 2  | loss: 1.02725 | train_auc: 0.53272 | valid_auc: 0.45416 |  0:00:00s
epoch 3  | loss: 0.95978 | train_auc: 0.5317  | valid_auc: 0.46482 |  0:00:00s
epoch 4  | loss: 0.95134 | train_auc: 0.52825 | valid_auc: 0.46802 |  0:00:00s
epoch 5  | loss: 0.95533 | train_auc: 0.53019 | valid_auc: 0.47228 |  0:00:00s
epoch 6  | loss: 0.96258 | train_auc: 0.53373 | valid_auc: 0.48134 |  0:00:00s
epoch 7  | loss: 0.91441 | train_auc: 0.53673 | valid_auc: 0.49627 |  0:00:00s
epoch 8  | loss: 0.93052 | train_auc: 0.5393  | valid_auc: 0.50533 |  0:00:00s
epoch 9  | loss: 0.85057 | train_auc: 0.54247 | valid_auc: 0.51812 |  0:00:01s
epoch 10 | loss: 0.84792 | train_auc: 0.54479 | valid_auc: 0.52665 |  0:00:01s
epoch 11 | loss: 0.80767 | train_auc: 0.54851 | valid_auc: 0.52985 |  0:00:01s
epoch 12 | loss: 0.82005 | train_auc: 0.54813 | valid_auc: 0.53785 |  0:00:01s
epoch 13 | loss: 0.84791 | train_auc: 0.55138 | vali

epoch 105| loss: 0.63714 | train_auc: 0.63889 | valid_auc: 0.60235 |  0:00:10s
epoch 106| loss: 0.65534 | train_auc: 0.64    | valid_auc: 0.60181 |  0:00:10s
epoch 107| loss: 0.65302 | train_auc: 0.6414  | valid_auc: 0.60128 |  0:00:10s
epoch 108| loss: 0.65729 | train_auc: 0.64237 | valid_auc: 0.59808 |  0:00:10s
epoch 109| loss: 0.65786 | train_auc: 0.64352 | valid_auc: 0.59915 |  0:00:10s
epoch 110| loss: 0.6498  | train_auc: 0.64193 | valid_auc: 0.59488 |  0:00:10s
epoch 111| loss: 0.66234 | train_auc: 0.6417  | valid_auc: 0.59808 |  0:00:10s
epoch 112| loss: 0.63517 | train_auc: 0.64205 | valid_auc: 0.59648 |  0:00:11s
epoch 113| loss: 0.65684 | train_auc: 0.64161 | valid_auc: 0.59701 |  0:00:11s
epoch 114| loss: 0.67401 | train_auc: 0.64109 | valid_auc: 0.60181 |  0:00:11s
epoch 115| loss: 0.63866 | train_auc: 0.64122 | valid_auc: 0.60501 |  0:00:11s
epoch 116| loss: 0.64437 | train_auc: 0.64264 | valid_auc: 0.60608 |  0:00:11s
epoch 117| loss: 0.63941 | train_auc: 0.64261 | vali

epoch 210| loss: 0.61685 | train_auc: 0.71593 | valid_auc: 0.63273 |  0:00:21s
epoch 211| loss: 0.59173 | train_auc: 0.71481 | valid_auc: 0.63273 |  0:00:21s
epoch 212| loss: 0.59153 | train_auc: 0.71506 | valid_auc: 0.63273 |  0:00:21s
epoch 213| loss: 0.60921 | train_auc: 0.71639 | valid_auc: 0.63539 |  0:00:21s
epoch 214| loss: 0.61612 | train_auc: 0.71842 | valid_auc: 0.63806 |  0:00:21s
epoch 215| loss: 0.59058 | train_auc: 0.71956 | valid_auc: 0.63539 |  0:00:21s
epoch 216| loss: 0.60639 | train_auc: 0.7207  | valid_auc: 0.6338  |  0:00:21s
epoch 217| loss: 0.60112 | train_auc: 0.72135 | valid_auc: 0.63486 |  0:00:21s
epoch 218| loss: 0.59969 | train_auc: 0.72294 | valid_auc: 0.63806 |  0:00:21s
epoch 219| loss: 0.60936 | train_auc: 0.72133 | valid_auc: 0.63593 |  0:00:22s
epoch 220| loss: 0.60694 | train_auc: 0.72077 | valid_auc: 0.63699 |  0:00:22s

Early stopping occurred at epoch 220 with best_epoch = 200 and best_valid_auc = 0.64819


/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [13]:
preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

BEST VALID SCORE FOR credit-g : 0.6481876332622601
FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [14]:
# save tabnet model
saving_path_name = "./tabnet_model_test_credit_g"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at ./tabnet_model_test_credit_g.zip


In [15]:
# define new model with basic parameters and load state dict weights
loaded_clf = TabNetClassifier()
loaded_clf.load_model("./tabnet_model_test_credit_g.zip")

/Users/nazaraburas/Desktop/dl_project/TabSec/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


# Predictions

In [16]:
dataset_name = 'credit-g'

preds = clf.predict_proba(X_test)
test_auc = roc_auc_score(y_score=preds[:,1], y_true=y_test)


preds_valid = clf.predict_proba(X_valid)
valid_auc = roc_auc_score(y_score=preds_valid[:,1], y_true=y_valid)

print("TabNet")
print(f"BEST VALID SCORE FOR {dataset_name} : {clf.best_cost}")
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

TabNet
BEST VALID SCORE FOR credit-g : 0.6481876332622601
FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [17]:
# check that best weights are used
assert np.isclose(valid_auc, np.max(clf.history['valid_auc']), atol=1e-6)

In [19]:
# sanity check
clf.predict(X_test)

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0])

In [20]:
# save tabnet model
saving_path_name = "./tabnet_model_credit_test_1"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at ./tabnet_model_credit_test_1.zip


In [21]:
loaded_preds = loaded_clf.predict_proba(X_test)
loaded_test_auc = roc_auc_score(y_score=loaded_preds[:,1], y_true=y_test)

print(f"FINAL TEST SCORE FOR {dataset_name} : {loaded_test_auc}")

FINAL TEST SCORE FOR credit-g : 0.6241426611796982


In [34]:
loaded_clf.predict(X_test)

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0])

# Generate Adversarial Examples

In [53]:
SEED = 11

settings = {'MaxIters': 500,
            'Alpha': 0.1,
            'Lambda': 8.5}

#settings['TestData'] = train[features].iloc[list(test_indices)]
settings['TestData'] = train.iloc[list(test_indices)]
settings['TestData'] = settings['TestData'].sample(n=10, random_state=SEED)
settings['TestDataOrig'] = settings['TestData']
settings['TestData'] = settings['TestData'][features]

# Sub sample

settings['Target'] = target
settings['FeatureNames'] = features
settings['Weights'] = weights
settings['Bounds'] = bounds


loaded_clf.predict(settings['TestData'].values)


array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0])

In [54]:
def gen_adv(model, config, method):
    extra_cols = ['orig_pred', 'adv_pred', 'iters']
    df_test = config['TestData']
    feature_names = config['FeatureNames']
    weights = config['Weights']
    bounds = config['Bounds']
    maxiters = config['MaxIters']
    alpha = config['Alpha']
    lambda_ = config['Lambda']
    target = config['Target']
    
    results = np.zeros((len(df_test), len(feature_names) + 1))
            
    i = -1
    n_samples = 0
    n_success = 0
    pert_norms = []
    weighted_pert_norms = []
    
    
    for _, row in tqdm(df_test.iterrows(), total=df_test.shape[0], desc="{}".format(method)):
        i += 1
        
        #x_tensor = torch.DoubleTensor(row[config['FeatureNames']])
        #x_tensor = torch.DoubleTensor(row.values)
        x_tensor = row.values
        
        n_samples += 1
        
        if method == 'LowProFool':
            orig_pred, adv_pred, x_adv, loop_i = lowProFool(x_tensor, model, weights, bounds,
                                                             maxiters, alpha, lambda_)
        elif method == 'Deepfool':
            x_tensor = torch.tensor(x_tensor)
            orig_pred, adv_pred, x_adv, loop_i = deepfool(x_tensor, model, maxiters, alpha,
                                                          bounds, weights=[])
        else:
            raise Exception("Invalid method", method)

        #pert = x_adv - x_tensor.numpy()
        pert = x_adv - x_tensor

        if orig_pred != adv_pred:
            n_success += 1
            pert_norms.append(np.linalg.norm(pert))
            weighted_pert_norms.append(np.linalg.norm(weights * pert))

        results[i] = np.append(x_adv, orig_pred)
    df = pd.DataFrame(results, index=df_test.index, columns=feature_names + [target])
    return df, n_success/n_samples, np.mean(weighted_pert_norms), np.std(weighted_pert_norms),\
                                    np.mean(pert_norms), np.std(pert_norms)


# Clipping function
def clip(current, low_bound, up_bound):
    assert(len(current) == len(up_bound) and len(low_bound) == len(up_bound))
    low_bound = torch.FloatTensor(low_bound)
    up_bound = torch.FloatTensor(up_bound)
    clipped = torch.max(torch.min(current, up_bound), low_bound)
    return clipped

# LowProFool

In [55]:
def lowProFool(x, model, weights, bounds, maxiters, alpha, lambda_):
    """
    Generates an adversarial examples x' from an original sample x

    :param x: tabular sample
    :param model: neural network
    :param weights: feature importance vector associated with the dataset at hand
    :param bounds: bounds of the datasets with respect to each feature
    :param maxiters: maximum number of iterations ran to generate the adversarial examples
    :param alpha: scaling factor used to control the growth of the perturbation
    :param lambda_: trade off factor between fooling the classifier and generating imperceptible adversarial example
    :return: original label prediction, final label prediction, adversarial examples x', iteration at which the class changed
    """

    r = Variable(torch.FloatTensor(1e-4 * np.ones(x.shape)), requires_grad=True) 
    v = torch.FloatTensor(np.array(weights))
    
    x_into_model = np.expand_dims(x, axis=0)
    x = torch.tensor(x)

    output = model.predict_proba(x_into_model) #outputs -> numpy array [[x, 1-x]]
    output = Variable(torch.tensor(output[0]), requires_grad=True)
    
    orig_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    target_pred = np.abs(1 - orig_pred)
    
    target = np.array([0., 1.]) if target_pred == 1 else np.array([1., 0.])
    target = Variable(torch.tensor(target, requires_grad=False)) 
    
    lambda_ = torch.tensor([lambda_])
    
    bce = nn.BCELoss()
    l1 = lambda v, r: torch.sum(torch.abs(v * r)) #L1 norm
    l2 = lambda v, r: torch.sqrt(torch.sum(torch.mul(v * r,v * r))) #L2 norm

    best_norm_weighted = np.inf
    best_pert_x = x
    
    loop_i, loop_change_class = 0, 0
    while loop_i < maxiters: 
        if r.grad is not None:
            r.grad.zero_()
            
        # make sure same type tensors
        output = output.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
        
        # Computing loss
        loss_1 = bce(output, target)
        loss_2 = l2(v, r)
        loss = (loss_1 + lambda_ * loss_2)

        # Get the gradient
        loss.backward(retain_graph=True)
        grad_r = r.grad.data.cpu().numpy().copy()

        # Guide perturbation to the negative of the gradient
        ri = - grad_r
    
        # limit huge step
        ri *= alpha

        # Adds new perturbation to total perturbation
        r = r.clone().detach().cpu().numpy() + ri
        
        # For later computation
        r_norm_weighted = np.sum(np.abs(r * weights))
        
        # Ready to feed the model
        r = Variable(torch.FloatTensor(r), requires_grad=True) 
        
        # Compute adversarial example
        xprime = x + r
    
        # Clip to stay in legitimate bounds
        xprime = clip(xprime, bounds[0], bounds[1])
        
        # model expects input of form [x1]
        xprime_into_model = torch.unsqueeze(xprime, 0)
        
        # Classify adversarial example
        output = model.predict_proba(xprime_into_model) #outputs -> numpy array [[x, 1-x]]
        
        output = Variable(torch.tensor(output[0]), requires_grad=True)
        
        output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
        
        # Keep the best adverse at each iterations
        if output_pred != orig_pred and r_norm_weighted < best_norm_weighted:
            best_r = r
            best_norm_weighted = r_norm_weighted
            best_pert_x = xprime

        if output_pred == orig_pred:
            loop_change_class += 1
            
        loop_i += 1 
        
    # Clip at the end no matter what
    best_pert_x = clip(best_pert_x, bounds[0], bounds[1])
    best_pert_x_into_model = torch.unsqueeze(best_pert_x, 0)
    output = model.predict_proba(best_pert_x_into_model)
    output = Variable(torch.tensor(output[0]), requires_grad=False)
    output_pred = output.max(0, keepdim=True)[1].cpu().numpy()
    
    return orig_pred, output_pred, best_pert_x.clone().detach().cpu().numpy(), loop_change_class

In [56]:
# Generate adversarial examples
df_adv_lpf, *lpf_data = gen_adv(loaded_clf, settings, 'LowProFool')

LowProFool: 100%|██████████| 10/10 [00:24<00:00,  2.47s/it]


# Attack Results

In [57]:
iterations = settings['MaxIters']
alpha = settings['Alpha']
lambdaa = settings['Lambda']
print(f'Iterations: {iterations} , Alpha: {alpha} , Lambda: {lambdaa}')
print('')
print('Before:')
print(loaded_clf.predict(settings['TestData'].values))
print('After:')
print(loaded_clf.predict(df_adv_lpf[features].values))
print('')


x = loaded_clf.predict(df_adv_lpf[features].values)

y = loaded_clf.predict(settings['TestData'].values)
result = sum(x != y) / len(x)
print(f"Adverserial generation success: {result}")

Iterations: 500 , Alpha: 0.1 , Lambda: 8.5

Before:
[1 1 1 0 1 0 0 0 1 0]
After:
[0 0 0 1 0 1 1 1 0 1]

Adverserial generation success: 1.0
